# 单发多框检测 / SSD / Single Shot Multibox Detection

* 基本网络用于从输入图像中提取特征

  （在分类层之前截断的VGG）

* 每个多尺度特征块将上一层提供的特征图的高和宽缩小
* 接近顶部的多尺度特征图较小，但具有较大的感受野，它们适合检测较少但较大的物体

阅读：单发多框检测\_SSD.pdf


<center><img src="../img/ssd.svg" width="55%"></center>

### 类别预测层

* $q$：目标类别的数量
* 锚框：$q+1$个类别（0类是背景）
* 类别预测层：一个保持输入高和宽的卷积层
* 输出通道数：$a(q+1)$

```
def cls_predictor(num_inputs, num_anchors, num_classes):
    return nn.Conv2d(num_inputs, 
        num_anchors * (num_classes + 1),
        kernel_size=3, padding=1)
```

### 边界框预测层

为每个锚框预测4个偏移量

```
def bbox_predictor(num_inputs, num_anchors):
    return nn.Conv2d(num_inputs, 
        num_anchors * 4, 
        kernel_size=3, padding=1)
```

### 高和宽减半块

* 应用VGG模块设计
* **两个**填充为$1$的$3\times3$的卷积层：不改变特征图的形状
* 步幅为$2$的$2\times2$最大池化层：将输入特征图的高度和宽度减少了一半
* receptive field：输出中的每个单元在输入上都有一个$6\times 6$的感受野

讨论：$6\times 6$的感受野

### 基本网络块

* 小的基础网络：串联3个高和宽减半块，并逐步将通道数翻倍
* 输入：$256\times256$，输出：$32 \times 32$

\begin{equation*}
\frac{256}{2^3}=32
\end{equation*}

### 完整的模型

* 完整的单发多框检测模型由五个模块组成
* 第一个是基本网络块
* 第二个到第四个是高和宽减半块
* 最后一个模块使用全局最大池将高度和宽度都降到1

* 一个较接近顶部的多尺度特征块是用于检测较大目标的

```
sizes = [[0.2, 0.272], [0.37, 0.447], 
         [0.54, 0.619], [0.71, 0.79],
         [0.88, 0.961]]
ratios = [[1, 2, 0.5]] * 5
```

* 锚框的个数：4

* 在所有五个尺度下，锚框总数：

\begin{equation*}
(32^2 + 16^2 + 8^2 + 4^2 + 1)\times 4 = 5444
\end{equation*}


### 定义损失函数和评价函数

* 有关锚框类别的损失：交叉熵损失函数
* 有关正类锚框偏移量的损失：$L_1$范数损失，即预测值和真实值之差的绝对值

观察：教材中的预测目标结果

阅读：单发多框检测\_SSD.pdf
